In [13]:
from scipy.linalg import expm
import numpy as np

import sys
sys.path.append('../')
import z_library.linear_system as ls

ls.set_print_form(precision=3)

In [14]:

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])
identity = np.array([[1, 0], [0, 1]])

mlist = [sigma_plus, sigma_minus, identity]

def tensor_product(matrices):
    result = np.array(mlist[matrices[0]])
    for matrix in matrices[1:]:
        result = np.kron(result, mlist[matrix])
    return result

In [15]:
def get_simulate_unitary(t,v):    
    # for unitary build contrary_v
    contrary_v = [not(i) for i in v]
    U = expm(-1j * 2 * np.pi * t * (tensor_product(v)+tensor_product(contrary_v)))
    return U

In [16]:
## find the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(get_simulate_unitary(0.8,[1,1,0]))
print(eigenvalues)

[0.309+0.951j 0.309-0.951j 1.   +0.j    1.   +0.j    1.   +0.j    1.   +0.j    1.   +0.j    1.   +0.j   ]


In [17]:
U= get_simulate_unitary(0.8,[1,1,0])


In [20]:
initvec = np.random.rand(8)
initvec = initvec / np.linalg.norm(initvec)
print(initvec)
print(U @ initvec)
## get density matrix
rho = np.outer(initvec, initvec.conj().T)
print(rho)


[0.188 0.23  0.469 0.293 0.454 0.014 0.457 0.437]
[0.188+0.j    0.071+0.434j 0.469+0.j    0.293+0.j    0.454+0.j    0.014+0.j    0.141+0.219j 0.437+0.j   ]
[[0.035 0.043 0.088 0.055 0.085 0.003 0.086 0.082]
 [0.043 0.053 0.108 0.067 0.104 0.003 0.105 0.101]
 [0.088 0.108 0.22  0.138 0.213 0.006 0.214 0.205]
 [0.055 0.067 0.138 0.086 0.133 0.004 0.134 0.128]
 [0.085 0.104 0.213 0.133 0.206 0.006 0.207 0.198]
 [0.003 0.003 0.006 0.004 0.006 0.    0.006 0.006]
 [0.086 0.105 0.214 0.134 0.207 0.006 0.209 0.2  ]
 [0.082 0.101 0.205 0.128 0.198 0.006 0.2   0.191]]


In [10]:
cir = QuantumCircuit(3)
cir.mcp(0.8, [0,1], 2)
new_cir = transpile(cir, basis_gates=['x','cp','y','z','cz','ccz','ccp','h'])
print(new_cir)

       ┌───┐     ┌───┐          ┌───┐   ┌───┐         
q_0: ──┤ H ├───■─┤ H ├─■────────┤ H ├─■─┤ H ├─■───────
       └───┘   │ └───┘ │        └───┘ │ └───┘ │       
q_1: ─■────────■───────┼──────────────■───────┼───────
      │P(0.4)          │P(-0.4)               │P(0.4) 
q_2: ─■────────────────■──────────────────────■───────
                                                      


In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.circuit.library import MCMT
from qiskit import transpile

# 输入qc, 返回电路酉矩阵
def get_circ_unitary(quantum_circuit):
  backend = Aer.get_backend('unitary_simulator' )
  new_circuit = transpile(quantum_circuit, backend)
  job = backend.run(new_circuit)
  result = job.result()
  unitary = result.get_unitary()
  return unitary

In [21]:
# 从q0加H后, cx控制其余门, 注意qiskit下标相反
from qiskit import QuantumCircuit
def iter_apply(qc,bitstring,i,num_qubits):
    flip = bitstring[0]==0
    if flip:
        bitstring = [not(j) for j in bitstring]
    if bitstring[1]:
        qc.x(num_qubits-i-1)
    qc.cx(num_qubits - i,num_qubits-(i+1))
    next_bitstring = bitstring[1:]
    if len(next_bitstring)>1:
        return iter_apply(qc,next_bitstring,i+1,num_qubits)
    else:
        return qc
        
    

def apply_convert(qc,bitstring):
    ## 1111
    num_qubits = len(bitstring)
    qc.x(num_qubits - 1)
    qc.h(num_qubits - 1)
    ## 1111+0111
    ##find bit start with 1
    iter_apply(qc,bitstring,1,num_qubits)

num_q = 3
qc = QuantumCircuit(num_q)
bitstring = [1,0,1]
for i in range(num_q-1):
    qc.x(i)
apply_convert(qc,bitstring)
qc.draw()

┌───┐          ┌───┐
q_0: ┤ X ├──────────┤ X ├
     ├───┤     ┌───┐└─┬─┘
q_1: ┤ X ├─────┤ X ├──■──
     ├───┤┌───┐└─┬─┘     
q_2: ┤ X ├┤ H ├──■───────
     └───┘└───┘

In [22]:
# 验证确实为 |101> + |010> 的叠加态
print(f'验证制备 的叠加态: {np.asarray(get_circ_unitary(qc))[:,0]}')


验证制备 的叠加态: [ 0.   +0.j  0.   +0.j  0.707+0.j  0.   +0.j  0.   +0.j -0.707-0.j  0.   +0.j  0.   +0.j]


In [26]:
def reversed_iter_apply(qc,bitstring,i,num_qubits):
    flip = bitstring[0]==0
    if flip:
        bitstring = [not(j) for j in bitstring]
    if len(bitstring)>1:
        reversed_iter_apply(qc,bitstring[1:],i+1,num_qubits)
    else:
        return qc
    qc.cx(num_qubits - i,num_qubits-(i+1))
    if bitstring[1]:
        qc.x(num_qubits-i-1)
    return qc
    

def reverse_apply_convert(qc,bitstring):
    ## 1111
    num_qubits = len(bitstring)
    reversed_iter_apply(qc,bitstring,1,num_qubits)
    qc.h(num_qubits - 1)
    qc.x(num_qubits - 1)
    ## 1111+0111
    ##find bit start with 1

num_q = 3
qc = QuantumCircuit(num_q)
bitstring = [1,0,1]
for i in range(num_q):
    qc.x(i)
reverse_apply_convert(qc,bitstring)
qc.draw()

┌───┐┌───┐               
q_0: ┤ X ├┤ X ├───────────────
     ├───┤└─┬─┘┌───┐          
q_1: ┤ X ├──■──┤ X ├──────────
     ├───┤     └─┬─┘┌───┐┌───┐
q_2: ┤ X ├───────■──┤ H ├┤ X ├
     └───┘          └───┘└───┘

In [109]:
print(f'验证反向的态: {np.asarray(get_circ_unitary(qc))[:,0]}')

验证反向的态: [ 0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j -0.707+0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.707+0.j  0.   +0.j  0.   +0.j]


In [79]:
def iter_apply(qc,bitstring,i,num_qubits):
    flip = bitstring[0]==0
    if flip:
        bitstring = [not(j) for j in bitstring]
    if bitstring[1]:
        qc.x(num_qubits-i-1)
    qc.cx(num_qubits - i,num_qubits-(i+1))
    next_bitstring = bitstring[1:]
    if len(next_bitstring)>1:
        return iter_apply(qc,next_bitstring,i+1,num_qubits)
    else:
        return qc

def apply_convert(qc,bitstring):
    num_qubits = len(bitstring)
    qc.x(num_qubits - 1)
    qc.h(num_qubits - 1)
    iter_apply(qc,bitstring,1,num_qubits)

def reversed_iter_apply(qc,bitstring,i,num_qubits):
    flip = bitstring[0]==0
    if flip:
        bitstring = [not(j) for j in bitstring]
    if len(bitstring)>1:
        reversed_iter_apply(qc,bitstring[1:],i+1,num_qubits)
    else:
        return qc
    qc.cx(num_qubits - i,num_qubits-(i+1))
    if bitstring[1]:
        qc.x(num_qubits-i-1)
    return qc
    

def reverse_apply_convert(qc,bitstring):
    ## 1111
    num_qubits = len(bitstring)
    reversed_iter_apply(qc,bitstring,1,num_qubits)
    qc.h(num_qubits - 1)
    qc.x(num_qubits - 1)
    ## 1111+0111
    ##find bit start with 1


def get_circuit(num_qubits,t,bitstring):
    qc = QuantumCircuit(num_qubits)
    # for i in range(num_qubits):
    #     if i == index:
    #         qc.x(i)
    reverse_apply_convert(qc,bitstring)
    qc.barrier(label='convert')
    qc.mcp(-2*np.pi * t, list(range(1,num_qubits)),0)
    qc.barrier(label='multi-ctrl')
    qc.x(num_qubits-1)
    
    qc.mcp(2*np.pi * t, list(range(1,num_qubits)),0)
    qc.x(num_qubits-1)
    qc.barrier(label='reverse')
    # qc.mcp(-2*np.pi * t, list(range(num_qubits-1)),num_qubits-1)
    apply_convert(qc,bitstring)

    # for i in range(num_qubits):
    #     if i == index:
    #         qc.x(i)
    return qc
t = 0.9
bitstring = [0,0,1,1]
num_q = len(bitstring)
qc =get_circuit(num_q,t,bitstring)

import numpy as np
unitary = get_circ_unitary(qc)
unitarymatrix= np.where(np.abs(unitary.data) < 1e-10, 0, unitary.data)
print(unitarymatrix.round(4))
print(qc.draw(fold=200))

[[ 1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.809-0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.588j  0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     1.   +0.j     0.   +0.j     0.   

In [80]:
np.allclose(unitarymatrix- get_simulate_unitary(t,bitstring), 0, atol=1e-10)

True

In [33]:
t =0.9
v = [0, 1, 0]

# for unitary build contrary_v
contrary_v = [not(i) for i in v]
U = expm(-1j * 2 * np.pi * t * (tensor_product(v)+tensor_product(contrary_v)))
U

array([[1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.809+0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.588j, 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.588j, 0.   +0.j   , 0.   +0.j   , 0.809+0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   ]])

In [52]:
def when_1_add_phase(theta):
    circ = QuantumCircuit(1)
    circ.unitary(np.array([[1, 0], [0, np.exp(1j * theta)]]), 0)
    gate = circ.to_gate()
    return gate
num_qubits = 3
t = 0.9
qc_ham_simu = QuantumCircuit(num_qubits)
gete1 = when_1_add_phase(2 * np.pi * t)
cc_gate1 = MCMT(gete1, num_qubits - 1, 1)
# 哪个是受控门无所谓, 全 1 加相位
qc_ham_simu.append(cc_gate1, range(num_qubits))
get_circ_unitary(qc_ham_simu)

Operator([[1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.809-0.588j]],
       

In [53]:
qc_ham= QuantumCircuit(num_qubits)
qc_ham.mcp(2*np.pi * t, list(range(1,num_qubits)),0)
get_circ_unitary(qc_ham)

Operator([[1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   ],
          [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.809-0.588j]],
       

In [54]:
np.allclose(get_circ_unitary(qc_ham_simu)- get_circ_unitary(qc_ham),0)

True

In [56]:
num_qubits = 3
qc_ham_simu = QuantumCircuit(num_qubits)

# ==== 酉矩阵开始
qc_ham_simu.cx(num_qubits - 2, num_qubits - 3)
qc_ham_simu.cx(num_qubits - 1, num_qubits - 2)
qc_ham_simu.h(num_qubits - 1)
qc_ham_simu.x(num_qubits - 1)

# 控制门开始

qc_ham_simu.barrier()
qc_ham_simu.mcp(-2*np.pi * t, list(range(1,num_qubits)),0)
qc_ham_simu.barrier()

qc_ham_simu.x(num_qubits - 1)
qc_ham_simu.mcp(2*np.pi * t, list(range(1,num_qubits)),0)
qc_ham_simu.x(num_qubits - 1)
qc_ham_simu.barrier()
# 控制门结束

qc_ham_simu.x(num_qubits - 1) 
qc_ham_simu.h(num_qubits - 1)
qc_ham_simu.cx(num_qubits - 1, num_qubits - 2)
qc_ham_simu.cx(num_qubits - 2, num_qubits - 3)
# ==== 酉矩阵结束
print(qc_ham_simu.draw(fold=200))
print(f'酉矩阵作用后:\n {np.asarray(get_circ_unitary(qc_ham_simu))}')

     ┌───┐                ░             ░                      ░                ┌───┐
q_0: ┤ X ├────────────────░──■──────────░───────■──────────────░────────────────┤ X ├
     └─┬─┘┌───┐           ░  │          ░       │              ░           ┌───┐└─┬─┘
q_1: ──■──┤ X ├───────────░──■──────────░───────■──────────────░───────────┤ X ├──■──
          └─┬─┘┌───┐┌───┐ ░  │P(-9π/5)  ░ ┌───┐ │P(9π/5) ┌───┐ ░ ┌───┐┌───┐└─┬─┘     
q_2: ───────■──┤ H ├┤ X ├─░──■──────────░─┤ X ├─■────────┤ X ├─░─┤ X ├┤ H ├──■───────
               └───┘└───┘ ░             ░ └───┘          └───┘ ░ └───┘└───┘          
酉矩阵作用后:
 [[ 1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.809-0.j     0.   +0.j     0.   +0.j     0.   +0.588j  0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j    

In [64]:
print(qc.draw(fold=200))

                    ┌───┐                          ┌───┐               
q_0: ───────────────┤ X ├─■───────────────■────────┤ X ├───────────────
               ┌───┐└─┬─┘ │               │        └─┬─┘┌───┐          
q_1: ──────────┤ X ├──■───■───────────────■──────────■──┤ X ├──────────
     ┌───┐┌───┐└─┬─┘      │P(-9π/5) ┌───┐ │P(9π/5) ┌───┐└─┬─┘┌───┐┌───┐
q_2: ┤ X ├┤ H ├──■────────■─────────┤ X ├─■────────┤ X ├──■──┤ H ├┤ X ├
     └───┘└───┘                     └───┘          └───┘     └───┘└───┘


In [57]:
num_qubits = 3
qc_ham_simu = QuantumCircuit(num_qubits)

# ==== 酉矩阵开始
qc_ham_simu.cx(num_qubits - 2, num_qubits - 3)
qc_ham_simu.cx(num_qubits - 1, num_qubits - 2)
qc_ham_simu.h(num_qubits - 1)
qc_ham_simu.x(num_qubits - 1)

# 控制门开始

qc_ham_simu.barrier()
# gate1: 对角阵 |0>态不动 |1>态加相位-e^it
gete1 = when_1_add_phase(-2 * np.pi * t)
cc_gate1 = MCMT(gete1, num_qubits - 1, 1)
# 哪个是受控门无所谓, 全 1 加相位
qc_ham_simu.append(cc_gate1, range(num_qubits))
qc_ham_simu.barrier()

# gate2: 对角阵 |0>态不动 |1>态加相位e^it
gete2 = when_1_add_phase(2 * np.pi * t)
cc_gate2 = MCMT(gete2, num_qubits - 1, 1)
qc_ham_simu.x(num_qubits - 1)
qc_ham_simu.append(cc_gate2, range(num_qubits))
qc_ham_simu.x(num_qubits - 1)
qc_ham_simu.barrier()
# 控制门结束

qc_ham_simu.x(num_qubits - 1) 
qc_ham_simu.h(num_qubits - 1)
qc_ham_simu.cx(num_qubits - 1, num_qubits - 2)
qc_ham_simu.cx(num_qubits - 2, num_qubits - 3)
# ==== 酉矩阵结束
print(qc_ham_simu.draw(fold=200))
print(f'酉矩阵作用后:\n {np.asarray(get_circ_unitary(qc_ham_simu))}')

     ┌───┐                ░ ┌───────┐ ░      ┌───────┐      ░                ┌───┐
q_0: ┤ X ├────────────────░─┤0      ├─░──────┤0      ├──────░────────────────┤ X ├
     └─┬─┘┌───┐           ░ │       │ ░      │       │      ░           ┌───┐└─┬─┘
q_1: ──■──┤ X ├───────────░─┤1 mcmt ├─░──────┤1 mcmt ├──────░───────────┤ X ├──■──
          └─┬─┘┌───┐┌───┐ ░ │       │ ░ ┌───┐│       │┌───┐ ░ ┌───┐┌───┐└─┬─┘     
q_2: ───────■──┤ H ├┤ X ├─░─┤2      ├─░─┤ X ├┤2      ├┤ X ├─░─┤ X ├┤ H ├──■───────
               └───┘└───┘ ░ └───────┘ ░ └───┘└───────┘└───┘ ░ └───┘└───┘          
酉矩阵作用后:
 [[ 1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.809+0.j     0.   +0.j     0.   +0.j    -0.   +0.588j  0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     1.   +0.j     0.   +

In [58]:
get_simulate_unitary(t,v)

array([[1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.809+0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.588j, 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.588j, 0.   +0.j   , 0.   +0.j   , 0.809+0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   ]])